## 5.1. Evaluation on public dataset

### 5.1.1. Original Model

In [1]:
! litgpt evaluate ../../M3_weightloading/trial_notebooks/intermediates/checkpoints/microsoft/phi-1_5 \
--tasks "mmlu_philosophy" \
--batch_size 4

{'access_token': None,
 'batch_size': 4,
 'checkpoint_dir': PosixPath('../../M3_weightloading/trial_notebooks/intermediates/checkpoints/microsoft/phi-1_5'),
 'device': None,
 'dtype': None,
 'force_conversion': False,
 'limit': None,
 'num_fewshot': None,
 'out_dir': None,
 'save_filepath': None,
 'seed': 1234,
 'tasks': 'mmlu_philosophy'}
{'checkpoint_dir': PosixPath('../../M3_weightloading/trial_notebooks/intermediates/checkpoints/microsoft/phi-1_5'),
 'output_dir': PosixPath('../../M3_weightloading/trial_notebooks/intermediates/checkpoints/microsoft/phi-1_5/evaluate')}
/Users/sharmilisrinivasan/miniconda3/envs/llm_from_scratch_py310/lib/python3.10/site-packages/litgpt/eval/evaluate.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/

### 5.1.2. Finetuned Model

In [2]:
! litgpt evaluate intermediates/my_finetuned_phi/final \
--tasks "mmlu_philosophy" \
--batch_size 4

{'access_token': None,
 'batch_size': 4,
 'checkpoint_dir': PosixPath('intermediates/my_finetuned_phi/final'),
 'device': None,
 'dtype': None,
 'force_conversion': False,
 'limit': None,
 'num_fewshot': None,
 'out_dir': None,
 'save_filepath': None,
 'seed': 1234,
 'tasks': 'mmlu_philosophy'}
{'checkpoint_dir': PosixPath('intermediates/my_finetuned_phi/final'),
 'output_dir': PosixPath('intermediates/my_finetuned_phi/final/evaluate')}
/Users/sharmilisrinivasan/miniconda3/envs/llm_from_scratch_py310/lib/python3.10/site-packages/litgpt/eval/evaluate.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This 

In [3]:
#  Footnote:  To list available evaluation tasks, run:
! litgpt evaluate list | grep -i mmlu

tinyMMLU
mmlu_pro_llama_business
mmlu_pro_llama_economics
mmlu_pro_llama_math
mmlu_pro_llama_chemistry
mmlu_pro_llama
mmlu_pro_llama_history
mmlu_pro_llama_health
mmlu_pro_llama_physics
mmlu_pro_llama_biology
mmlu_pro_llama_computer_science
mmlu_pro_llama_law
mmlu_pro_llama_philosophy
mmlu_pro_llama_engineering
mmlu_pro_llama_psychology
mmlu_pro_llama_other
mmlu_llama_prehistory
mmlu_llama_humanities_tasks
mmlu_llama_econometrics
mmlu_llama_social_sciences_tasks
mmlu_llama_management
mmlu_llama_other_tasks
mmlu_llama_sociology
mmlu_llama_college_chemistry
mmlu_llama_stem_tasks
mmlu_llama_high_school_statistics
mmlu_llama_professional_psychology
mmlu_llama_social_sciences
mmlu_llama_machine_learning
mmlu_llama_high_school_chemistry
mmlu_llama_college_mathematics
mmlu_llama_high_school_geography
mmlu_llama_moral_disputes
mmlu_llama_high_school_mathematics
mmlu_llama_humanities
mmlu_llama_high_school_microeconomics
mmlu_llama_computer_security
mmlu_llama_miscellaneous
mmlu_llama_conceptua

## 5.2. Evaluation using LLM as Judge

In [4]:
from litgpt import LLM

/Users/sharmilisrinivasan/miniconda3/envs/llm_from_scratch_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 5.2.1. Load LLMJudge Model

In [ ]:
# ! cd intermediates
! litgpt download meta-llama/Llama-3.2-1B-Instruct --access_token=<HF Access Token>o
# ! cd ..

### 5.2.2. Load Data

In [5]:
import json
from tqdm import tqdm

In [6]:
with open("intermediates/test_with_before_after_responses.json", "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))
print("First entry:", json_data[0])

Number of entries: 165
First entry: {'instruction': 'Rewrite the sentence using a simile.', 'input': 'The car is very fast.', 'output': 'The car is as fast as lightning.', 'base_model': '\nThe car is like a cheetah, quick and speedy.\n\n### Output:\n```python\n# Solution will be provided in a separate response\n```\n\nThese exercises are designed to help you grasp the nuance in sim', 'finetuned_model': ' graffiti signature clim functionchart platinum thankfulCamucked organic cancelled lockedral steady inspiredGold suggested rend refres prox guideline gold lipstick504 profitable awayFFFF ruby coarse feedbackFeed trait missing benefit prosper tf lipstickjack happen grateful influentialC wouldx Fault admired[_ inject affucks'}


### 5.2.3. Utilities for LLM as judge

In [7]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

print("Formatted input for first entry:", format_input(json_data[0]))

Formatted input for first entry: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence using a simile.

### Input:
The car is very fast.


In [8]:
import re

def judge_using_llm(json_data, json_key_response):
    to_return_scores = []
    
    evaluation_llm = LLM.load("intermediates/checkpoints/meta-llama/Llama-3.2-1B-Instruct")

    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key_response]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Your responses should strictly follow the format below:"
            f"Score: [Insert score here]"
            f"Adhere to the following instructions to complete the task:"
            f"1. You will fail your task if the output generated does not follow the format mentioned above."
            f"2. You will fail your task if the output generated contains any other text other than the score."
            f"3. Strictly Do not add Instruction or Input or Express to the output."
        )
        score = evaluation_llm.generate(prompt, max_new_tokens=50)
        try:
            match = re.search(r"Score:[\s]*(\d+)", score)
            if match:
                score = match.group(1)
                to_return_scores.append(int(score))
            else:
                print(f"No match found: {score}")
        except ValueError:
            print(f"ValueError: {score}")
            continue

    del evaluation_llm
    
    return to_return_scores

### 5.2.4. Score Base and Finetuned Models

#### 5.2.4.1. Score of the base model

In [9]:
base_model_scores = judge_using_llm(json_data, "base_model")
print("# Base model scores:", len(base_model_scores))
print("Overall Average Base Model Score:", sum(base_model_scores)/ len(base_model_scores))

Scoring entries:   1%|          | 1/165 [00:10<28:54, 10.57s/it]

No match found: ## Step 1: Analyze the given instruction and the provided options
The instructions require rewriting a sentence using a simile and providing a score between 0 and 100. We have three options for the sentence.
 
The car is veryfast


Scoring entries:   4%|▎         | 6/165 [00:40<16:31,  6.24s/it]

No match found: ## Step 1: Determine the correct score for the given instruction
The instruction was delivered clearly. The score: 100.


Scoring entries:   5%|▌         | 9/165 [01:04<19:35,  7.53s/it]

...

No match found: ## Step 1: Understand the instruction
The task requires rewriting the given text using a simile to make it more engaging and readable.

## Step 2: Rewrite the sentence using a simile
The original sentence "The cake is very sweet


Scoring entries:  99%|█████████▉| 164/165 [12:47<00:04,  4.57s/it]

No match found: Score: 

A synonym for 'clever' is 'smart.'


Scoring entries: 100%|██████████| 165/165 [12:50<00:00,  4.67s/it]

# Base model scores: 112
Overall Average Base Model Score: 47.919642857142854


#### 5.2.4.2. Score of the finetuned model

In [10]:
finetuned_model_scores = judge_using_llm(json_data, "finetuned_model")
print("# Finetuned model scores:", len(finetuned_model_scores))
print("Overall Average Finetuned Model Score:", sum(finetuned_model_scores) / len(finetuned_model_scores))

Scoring entries:   1%|          | 2/165 [00:08<10:22,  3.82s/it]

No match found: 15


Scoring entries:   2%|▏         | 3/165 [00:10<08:39,  3.21s/it]

No match found: Score: 

Mrs. Bennet


Scoring entries:   3%|▎         | 5/165 [00:16<08:05,  3.04s/it]

...

No match found: Milliliters in a liter (ml to L) conversion table:


Scoring entries:  99%|█████████▉| 163/165 [08:00<00:05,  2.88s/it]

No match found: Sending


Scoring entries: 100%|██████████| 165/165 [08:03<00:00,  2.93s/it]

No match found: score:10
# Finetuned model scores: 94
Overall Average Finetuned Model Score: 41.340425531914896
